# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data 

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보 
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise 

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [16]:
data = pd.read_csv("assignment3_creditcard.csv")

In [17]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [18]:
y = data['Class']    # 타겟 변수
X = data.drop(columns='Class')

In [59]:
X.describe()    # 모두 평균이 0에 가까우므로 추가로 변환하지 않음.

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
count,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,...,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000
mean,-0.015438,0.053653,-0.046031,0.037348,-0.033724,-0.003299,-0.051054,0.006064,-0.018530,-0.041149,...,0.007183,0.002633,0.010289,-0.000656,-0.004800,-0.000897,-0.001989,-0.000765,0.000948,0.001535
std,2.031529,1.616186,1.758169,1.482109,1.486998,1.339259,1.454827,1.364342,1.134065,1.252593,...,0.827420,0.720307,0.847152,0.739469,0.593663,0.603349,0.517968,0.483852,0.397075,0.296736
min,-30.552380,-42.172688,-31.103685,-5.560118,-42.147898,-21.929312,-41.506796,-39.267378,-13.434066,-24.403185,...,-4.619034,-21.387122,-21.453736,-8.887017,-36.666000,-2.718024,-6.712624,-2.241620,-7.418878,-9.617915
25%,-0.916927,-0.575381,-0.899872,-0.843321,-0.714901,-0.763757,-0.568146,-0.206103,-0.661909,-0.543450,...,-0.459073,-0.209678,-0.225520,-0.539244,-0.160583,-0.356047,-0.318619,-0.327343,-0.070558,-0.052189
50%,0.020050,0.075358,0.180610,-0.008844,-0.060040,-0.271363,0.036107,0.022463,-0.055095,-0.097390,...,0.002111,-0.062792,-0.028778,0.007302,-0.011199,0.040006,0.019770,-0.056260,0.002049,0.011075
75%,1.317461,0.806957,1.029928,0.771958,0.613328,0.397269,0.559409,0.329606,0.605704,0.460681,...,0.467781,0.131199,0.184312,0.526358,0.146835,0.437146,0.352717,0.240713,0.091637,0.078911
max,2.399484,21.467203,4.069865,11.927512,32.911462,22.529298,36.677268,20.007208,8.113152,15.236028,...,5.228342,26.237391,27.202839,8.361985,9.637187,3.948061,2.510401,3.122747,11.135740,14.929133


### 시도 1

In [54]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=2023)

print(train_X.shape, test_X.shape)

(21508, 28) (7170, 28)


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

model = LogisticRegression()
model.fit(train_X, train_y)

y_pred = model.predict(test_X)
# print(model.predict(test_X))

print('acc:', model.score(test_X, test_y))
print('precision:', precision_score(test_y, y_pred))
print('recall:', recall_score(test_y, y_pred))
print('f1:', f1_score(test_y, y_pred))

confusion_matrix(test_y, y_pred)

acc: 0.9974895397489539
precision: 1.0
recall: 0.7142857142857143
f1: 0.8333333333333333


array([[7107,    0],
       [  18,   45]], dtype=int64)

In [56]:
data['Class'].value_counts()

0    28432
1      246
Name: Class, dtype: int64

- 데이터 불균형이 심하여 class '1'에 대한 예측 성능이 좋지 않게 나옴
- precision은 높지만, recall이 낮음
- f1 score를 높이는 방향으로 수정

### 시도 2

In [57]:
from sklearn.model_selection import train_test_split

# 데이터 분리 시, stratify 매개변수 사용
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=2023, stratify=y)

print(train_X.shape, test_X.shape)

(21508, 28) (7170, 28)


In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

model = LogisticRegression()
model.fit(train_X, train_y)

y_pred = model.predict(test_X)
# print(model.predict(test_X))

print('acc:', model.score(test_X, test_y))
print('precision:', precision_score(test_y, y_pred))
print('recall:', recall_score(test_y, y_pred))
print('f1:', f1_score(test_y, y_pred))

confusion_matrix(test_y, y_pred)

acc: 0.997629009762901
precision: 0.8947368421052632
recall: 0.8225806451612904
f1: 0.8571428571428571


array([[7102,    6],
       [  11,   51]], dtype=int64)

- precision은 조금 낮아졌으나, 그에 비해 recall이 더 높아져서 f1 score가 증가하였다.
- 데이터 불균형 시, 타겟 클래스의 분포를 반영한 데이터 분리가 중요해보인다.

### 시도 3

In [61]:
# 언더 샘플링

class_0 = data[data['Class']==0]
class_1 = data[data['Class']==1]

class_0 = class_0.sample(n=15000, random_state=2023)

data2 = pd.concat([class_0, class_1], axis=0)
data2['Class'].value_counts()

0    15000
1      246
Name: Class, dtype: int64

In [62]:
y = data2['Class']    # 타겟 변수
X = data2.drop(columns='Class')

In [63]:
from sklearn.model_selection import train_test_split

# 데이터 분리 시, stratify 매개변수 사용
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=2023, stratify=y)

print(train_X.shape, test_X.shape)

(11434, 28) (3812, 28)


In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

model = LogisticRegression()
model.fit(train_X, train_y)

y_pred = model.predict(test_X)
# print(model.predict(test_X))

print('acc:', model.score(test_X, test_y))
print('precision:', precision_score(test_y, y_pred))
print('recall:', recall_score(test_y, y_pred))
print('f1:', f1_score(test_y, y_pred))

confusion_matrix(test_y, y_pred)

acc: 0.9968520461699895
precision: 0.9807692307692307
recall: 0.8225806451612904
f1: 0.8947368421052632


array([[3749,    1],
       [  11,   51]], dtype=int64)

- f1 score는 높아졌으나, recall은 그대로 이고 class 1 예측 성능도 큰 변화가 없었다.